## Python traceback库详解

traceback模块提供了一个标准接口来提取、格式化和打印程序的堆栈跟踪信息。它精确地模拟了Python解释器在打印堆栈跟踪时的行为。

刚接触Python的时候，简单的异常处理已经可以帮助我们解决大多数问题，但是随着逐渐地深入，我们会发现有很多情况下简单的异常处理（通常是异常的value值），只有很少量的信息，我们很难定位在哪块代码出的问题，以及如何出现这种异常。那么到底要如何打印更加详细的信息呢？

这时，就是traceback模块大施拳脚的时候了！


Python程序的traceback信息均来源于一个叫做traceback object的对象，而这个traceback object通常是通过函数sys.exc_info()来获取的

exc_type, exc_value, exc_traceback_obj = sys.exc_info() 

可以看出，sys.exc_info()获取了当前处理的exception的相关信息，并返回一个元组，元组的第一个数据是异常的类型(示例是NameError类型)，第二个返回值是异常的value值，第三个就是我们要的traceback object.有了traceback object我们就可以通过traceback module来打印和格式化traceback的相关信息。

Python的traceback module提供一整套接口用于提取，格式化和打印Python程序的stack traces信息，下面我们通过例子来详细了解下这些接口：

1、traceback.print_tb(tb[, limit[, file]])

tb: 这个就是traceback object, 是我们通过sys.exc_info获取到的

limit: 这个是限制stack trace层级的，如果不设或者为None，就会打印所有层级的stack trace

file: 这个是设置打印的输出流的，可以为文件，也可以是stdout之类的file-like object。如果不设或为None，则输出到sys.stderr。

2、traceback.print_exception(etype, value, tb[, limit[, file]])

跟print_tb相比多了两个参数etype和value，分别是exception type和exception value，加上tb(traceback object)，正好是sys.exc_info()返回的三个值

另外，与print_tb相比，打印信息多了开头的"Traceback (most...)"信息以及最后一行的异常类型和value信息

还有一个不同是当异常为SyntaxError时，会有"^"来指示语法错误的位置


3、traceback.print_exc([limit[, file]])

print_exc是简化版的print_exception, 由于exception type, value和traceback object都可以通过sys.exc_info()获取，因此print_exc()就自动执行exc_info()来帮助获取这三个参数了，也因此这个函数是我们的程序中最常用的，因为它足够简单

4、traceback.format_exc([limit[, file]])

有时候我们想得到的是一个字符串，比如我们想通过logger将异常记录在log里，这个时候就需要format_exc了，这个也是最常用的一个函数，它跟print_exc用法相同，只是不直接打印而是返回了字符串。

In [2]:
import sys, traceback

def lumberjack():
    bright_side_of_death()

def bright_side_of_death():
    return tuple()[0]

try:
    lumberjack()
except IndexError:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print("*** print_tb:")
    traceback.print_tb(exc_traceback, limit=1, file=sys.stdout)
    print("*** print_exception:")
    traceback.print_exception(exc_type, exc_value, exc_traceback,
                              limit=2, file=sys.stdout)
    print("*** print_exc:")
    traceback.print_exc(limit=2, file=sys.stdout)
    print("*** format_exc, first and last line:")
    formatted_lines = traceback.format_exc().splitlines()
    print(formatted_lines[0])
    print(formatted_lines[-1])
    print("*** format_exception:")
    print(repr(traceback.format_exception(exc_type, exc_value,
                                          exc_traceback)))
    print("*** extract_tb:")
    print(repr(traceback.extract_tb(exc_traceback)))
    print("*** format_tb:")
    print(repr(traceback.format_tb(exc_traceback)))
    print("*** tb_lineno:", exc_traceback.tb_lineno)

*** print_tb:
  File "<ipython-input-2-97ab824f2cc0>", line 10, in <module>
    lumberjack()
*** print_exception:
Traceback (most recent call last):
  File "<ipython-input-2-97ab824f2cc0>", line 10, in <module>
    lumberjack()
  File "<ipython-input-2-97ab824f2cc0>", line 4, in lumberjack
    bright_side_of_death()
IndexError: tuple index out of range
*** print_exc:
Traceback (most recent call last):
  File "<ipython-input-2-97ab824f2cc0>", line 10, in <module>
    lumberjack()
  File "<ipython-input-2-97ab824f2cc0>", line 4, in lumberjack
    bright_side_of_death()
IndexError: tuple index out of range
*** format_exc, first and last line:
Traceback (most recent call last):
IndexError: tuple index out of range
*** format_exception:
['Traceback (most recent call last):\n', '  File "<ipython-input-2-97ab824f2cc0>", line 10, in <module>\n    lumberjack()\n', '  File "<ipython-input-2-97ab824f2cc0>", line 4, in lumberjack\n    bright_side_of_death()\n', '  File "<ipython-input-2-97ab824f2c